# Extract names from doc - Syntaxnet

In [1]:
# Import libraries
import pandas as pd
import re
import csv

Adjust scroll bar activation threshold...

In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 100;

<IPython.core.display.Javascript object>

In [3]:
# Choose agreement file from list in csv

# read csv file
agreements_data = pd.read_csv("data/agreements_dataset1.csv")
print "\n","Agreements data read successfully!"

# Select text file name.  4 is a simple sentence
selected_text_file_name = agreements_data['file_name'][19]
print "Selected text file name from csv -> ", selected_text_file_name, "\n"


Agreements data read successfully!
Selected text file name from csv ->  Cloudfind Limited_eula.txt 



In [4]:
# Cleanse and parse text into separate sentences

caps = "([A-Z])"
prefixes = "(Mr|St|Mrs|Ms|Dr|www|1|2|2|3|4|5|6|7|8|9|10)[.]"
suffixes = "(Inc|INC|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = text.replace("\r"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + caps + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(caps + "[.]" + caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + caps + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

filepath = "data/" + selected_text_file_name
with open(filepath, 'r') as my_agreement:
    my_agreement_text=my_agreement.read()

my_agreement_lines = split_into_sentences(my_agreement_text)
print "\n", "Agreement text has been cleansed and parsed into separate sentences.", "\n"


Agreement text has been cleansed and parsed into separate sentences. 



In [5]:
# Find sentences with "between"

key_sentences = []
type_of_sentence = ""
key_words = ["between"]
Party1 = "not found yet"
Party2 = "not found yet"

for line in my_agreement_lines:
    for word in key_words:
        if word in line.lower(): 
            key_sentences.append(line)
                
print "\n", "Number of sentences retrieved with the word 'between':", len(key_sentences),  "\n"


Number of sentences retrieved with the word 'between': 2 



In [6]:
# Find key sentence(s)

def pre_process_sentence(key_sentences):

    with open("data/syntaxnet_processing.txt", "w") as text_file:
        text_file.write(key_sentences[0])

if len(key_sentences) == 0: 
    print "There is no 'between' clause in the document."
    print "Party1 ->",Party1
    print "Party2 ->",Party2
    
if len(key_sentences) == 1: pre_process_sentence(key_sentences) 
    
if len(key_sentences) > 1: 
    
    key_sentences2 = []
    listOfWords = ['between','agreement']

    for line in key_sentences:
        if all(word in line.lower() for word in listOfWords):
            key_sentences2.append(line)
    
    if len(key_sentences2) == 0: print "no key sentences returned." 
    if len(key_sentences2) == 1: pre_process_sentence(key_sentences2)
    if len(key_sentences2) > 1:
        print "Number of sentences retrieved:", len(key_sentences2)
        print "Processing the first retrieved sentence"
        pre_process_sentence(key_sentences2)

Number of sentences retrieved: 2
Processing the first retrieved sentence


In [ ]:
%%bash
syntaxnet/demo_carl2.sh < data/syntaxnet_processing.txt > data/syntaxnet_processing_output.txt

In [8]:
# read from Syntaxnet output tab delimited textfile

syntaxnet_array = []

def readata(filename):
      file=open(filename,'r')
      lines=file.readlines()
      lines=lines[:-1]
      sentence_data_syntaxnet=csv.reader(lines,delimiter='\t')
      syntaxnet_array=list(sentence_data_syntaxnet)
      return (syntaxnet_array)
                    
syntaxnet_array = readata("data/syntaxnet_processing_output.txt")

In [9]:
# convert text file to CSV (for debugging purposes)

txt_file = r"data/syntaxnet_processing_output.txt"
csv_file = r"data/syntaxnet_processing_output.csv"

in_txt = csv.reader(open(txt_file, "rb"), delimiter = '\t')
out_csv = csv.writer(open(csv_file, 'wb'))

out_csv.writerows(in_txt)

In [10]:
# Find party1 and party2

# Print results
def extract_result(Party1, Party2):
    print "\n", "Agreement:", selected_text_file_name
    print "Party1:",Party1,",","Party2:",Party2,"\n"
 
def process_sentence(syntaxnet_array):
    
    # Locate "between"
    array_count = 0
    for i in syntaxnet_array:
        array_count += 1
        if i[1].lower() == "between": break
    
    # Capture word after 'between'
    Party1 = syntaxnet_array[array_count][1]
        
    # Check for companies with multiple words
    array_count += 1
    while syntaxnet_array[array_count][3] == "NOUN":
        Party1 = Party1 + " " + syntaxnet_array[array_count][1]
        array_count += 1
        
    # Locate "and"
    array_count2 = array_count
    for i in syntaxnet_array:
        if syntaxnet_array[array_count2][1].lower() == "and": 
            array_count2 += 1
            break
        array_count2 += 1
    
    # Capture word after 'between'
    Party2 = syntaxnet_array[array_count2][1]
        
    # Check for companies with multiple words
    array_count2 += 1
    while syntaxnet_array[array_count2][3] == "NOUN":
        Party2 = Party2 + " " + syntaxnet_array[array_count2][1]
        array_count2 += 1
    
    print "\n", "Agreement:", selected_text_file_name
    print "Party1:",Party1,",","Party2:",Party2,"\n"
    
process_sentence(syntaxnet_array)


Agreement: Cloudfind Limited_eula.txt
Party1: you , Party2: Cloudfind Limited 



## Previous results

**Agreement**: Google_Construction_Agreement_C.txt <br>
**Party1**: GOOGLE INC. , **Party2**: S.G. Cunningham 

**Agreement**: 2Think1 Solutions Inc_eula.txt <br>
**Party1**: you , **Party2**: 2THINK1 SOLUTIONS INC

**Agreement**: ABBYY_eula.txt <br>
**Party1**: you , **Party2**: ABBYY 

**Agreement**: Aeria Games & Entertainment Inc_eula.txt <br>
**Party1**: YOU , **Party2**: AGE 

**Agreement**: AllCursors_eula.txt <br>
**Party1**: you , **Party2**: Licensor 

**Agreement**: ALM Works Ltd_eula.txt **(FAIL)** <br>
**Party1**: you , **Party2**: the 

**Agreement**: AnyChart_eula.txt <br>
**Party1**: you , **Party2**: Sibental 

**Agreement**: AOL Inc_eula.txt <br>
**Party1**: you , **Party2**: us 

**Agreement**: app square OG_eula.txt <br>
**Party1**: you , **Party2**: appsquare OG

**Agreement**: APPEARTOME LIMITED_eula.txt <br>
**Party1**: you , **Party2**: Appeartome Limited 

**Agreement**: Atlassian_eula.txt <br>
**Party1**: you , **Party2**: Atlassian Pty Ltd 

**Agreement**: Avanquest Software SA_eula.txt <br>
**Party1**: you , **Party2**: Avanquest Software S.A. 

**Agreement**: Bitdefender_eula.txt <br>
**Party1**: you , **Party2**: BITDEFENDER 

**Agreement**: Blizzard Entertainment Inc_eula.txt **(FAIL)** <br>
**Party1**: the parties , **Party2**: supersedes

**Agreement**: BTC_eula.txt <br>
**Party1**: you , **Party2**: bigtincan 

**Agreement**: Caphyon_eula.txt <br>
**Party1**: YOU , **Party2**: CAPHYON 

**Agreement**: CareerBuilder_eula.txt <br>
**Party1**: you , **Party2**: CareerBuilder 

**Agreement**: Caristix_eula.txt <br>
**Party1**: you , **Party2**: Caristix 

**Agreement**: ChemAxon Ltd_eula.txt <br>
**Party1**: you , **Party2**: ChemAxon Ltd.

**Agreement**: Cloudfind Limited_eula.txt <br>
**Party1**: you , **Party2**: Cloudfind Limited

**Agreement**: Concurrent Inc_eula.txt <br>
**Party1**: Concurrent , **Party2**: you 